# Import

In [2]:
import pandas as pd
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.neural_network import MLPClassifier

In [5]:
train_df = pd.read_csv("../train.csv")
test_df = pd.read_csv("../test.csv")


In [6]:
train_df

,id,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,0,15674932,Okwudilichukwu,668,France,Male,33.0,3,0.00,2,1.0,0.0,181449.97,0
1,1,15749177,Okwudiliolisa,627,France,Male,33.0,1,0.00,2,1.0,1.0,49503.50,0
2,2,15694510,Hsueh,678,France,Male,40.0,10,0.00,2,1.0,0.0,184866.69,0
3,3,15741417,Kao,581,France,Male,34.0,2,148882.54,1,1.0,1.0,84560.88,0
4,4,15766172,Chiemenam,716,Spain,Male,33.0,5,0.00,2,1.0,1.0,15068.83,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
165029,165029,15667085,Meng,667,Spain,Female,33.0,2,0.00,1,1.0,1.0,131834.75,0
165030,165030,15665521,Okechukwu,792,France,Male,35.0,3,0.00,1,0.0,0.0,131834.45,0
165031,165031,15664752,Hsia,565,France,Male,31.0,5,0.00,1,1.0,1.0,127429.56,0
165032,165032,15689614,Hsiung,554,Spain,Female,30.0,7,161533.00,1,0.0,1.0,71173.03,0


In [22]:
def split(df, title):
    return df.drop(title, axis=1), df[title]
def getDummies(df):
    object_df = df.select_dtypes(include=['object'])
    object_columns = object_df.columns
    return pd.get_dummies(object_df, prefix=object_columns), object_columns
def transform(df, dummies, columns, all_columns):
    return pd.concat([df.drop(columns, axis=1), dummies.reindex(columns=all_columns, fill_value=False)], axis=1)
def unionColumns(column_a, column_b):
    return list(set(column_a).union(column_b))

In [31]:
train_df.select_dtypes(include='object')

,Surname,Geography,Gender
0,Okwudilichukwu,France,Male
1,Okwudiliolisa,France,Male
2,Hsueh,France,Male
3,Kao,France,Male
4,Chiemenam,Spain,Male
...,...,...,...
165029,Meng,Spain,Female
165030,Okechukwu,France,Male
165031,Hsia,France,Male
165032,Hsiung,Spain,Female


In [35]:
(train_dummies, train_columns), (test_dummies, test_columns) = getDummies(train_df), getDummies(test_df)
all_columns = unionColumns(train_dummies.columns, test_dummies.columns)
transformed_train = transform(train_df, train_dummies, train_columns, all_columns)
transformed_test = transform(test_df, test_dummies, test_columns, all_columns)

In [36]:
train_x, train_y = split(transformed_train, "Exited")

In [ ]:
catboost = CatBoostClassifier(has_time=True)
catboost.fit(train_x, train_y)

In [39]:
catboost.predict_proba(transformed_test)

array([[0.98295331, 0.01704669],
       [0.19489195, 0.80510805],
       [0.98065279, 0.01934721],
       ...,
       [0.98777648, 0.01222352],
       [0.86099175, 0.13900825],
       [0.80114961, 0.19885039]])